In [1]:
!nvidia-smi
import torch
print("torch:", torch.__version__)
print("cuda:", torch.cuda.is_available())


Sat Feb  7 03:24:02 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip -q install kaggle

In [ ]:
from google.colab import files
files.upload()  # upload kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp bdd100k-seg-6cls-api.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!ls -la ~/.kaggle


In [ ]:
DATASET_SLUG = "manhdung4869/bdd100k-seg-6class-custom"

!kaggle datasets download -d {DATASET_SLUG} -p /content --force
!ls -lh /content


In [ ]:
import glob, os
zip_files = glob.glob("/content/*.zip")
print("zip_files:", zip_files)

for z in zip_files:
    !unzip -q -o "$z" -d /content/dataset

!find /content/dataset -maxdepth 2 -type d | head -n 50


In [ ]:
from pathlib import Path

DATA_ROOT = Path("/content/dataset/bdd_seg_6cls_final")
assert DATA_ROOT.exists(), f"DATA_ROOT not found: {DATA_ROOT}"
print("DATA_ROOT:", DATA_ROOT)

def count_files(p: Path, exts):
    if not p.exists(): return 0
    return sum(1 for f in p.iterdir() if f.is_file() and f.suffix.lower() in exts)

IMG_EXTS = {".jpg",".jpeg",".png"}
for sp in ["train","val","test"]:
    ni = count_files(DATA_ROOT/"images"/sp, IMG_EXTS)
    nm = count_files(DATA_ROOT/"masks"/sp, {".png"})
    print(f"{sp:>5}: images={ni} masks={nm}")


In [ ]:
!pip -q install pillow


In [ ]:
import random, time, gc
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms.functional as TF

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)

class BDDSeg6Cls(Dataset):
    def __init__(self, root: Path, split: str, img_size: int = 512, augment: bool = False):
        self.root = Path(root)
        self.split = split
        self.img_dir = self.root / "images" / split
        self.msk_dir = self.root / "masks" / split
        self.img_size = img_size
        self.augment = augment

        self.images = sorted([p for p in self.img_dir.iterdir()
                              if p.is_file() and p.suffix.lower() in {".jpg",".jpeg",".png"}])
        self.mask_index = {p.stem.lower(): p for p in self.msk_dir.glob("*.png")}

        # keep only paired
        self.images = [p for p in self.images if p.stem.lower() in self.mask_index]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        msk_path = self.mask_index[img_path.stem.lower()]

        img = Image.open(img_path).convert("RGB")
        mask = Image.open(msk_path)

        # resize (no caching)
        img = TF.resize(img, [self.img_size, self.img_size], interpolation=TF.InterpolationMode.BILINEAR)
        mask = TF.resize(mask, [self.img_size, self.img_size], interpolation=TF.InterpolationMode.NEAREST)

        # light aug
        if self.augment:
            if random.random() < 0.5:
                img = TF.hflip(img)
                mask = TF.hflip(mask)

        img_t = TF.to_tensor(img)
        img_t = TF.normalize(img_t, mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])

        mask_t = torch.from_numpy(np.array(mask, dtype=np.uint8)).long()
        return img_t, mask_t

IMG_SIZE = 512   # RAM/GPU-safe
BATCH = 4        # safe for T4 / P100
NW = 0           # RAM-safe

train_ds = BDDSeg6Cls(DATA_ROOT, "train", img_size=IMG_SIZE, augment=True)
val_ds   = BDDSeg6Cls(DATA_ROOT, "val",   img_size=IMG_SIZE, augment=False)
test_ds  = BDDSeg6Cls(DATA_ROOT, "test",  img_size=IMG_SIZE, augment=False)

train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True,  num_workers=NW, pin_memory=False)
val_loader   = DataLoader(val_ds,   batch_size=BATCH, shuffle=False, num_workers=NW, pin_memory=False)
test_loader  = DataLoader(test_ds,  batch_size=BATCH, shuffle=False, num_workers=NW, pin_memory=False)

print("sizes:", len(train_ds), len(val_ds), len(test_ds))

# mask sanity
p = next((DATA_ROOT/"masks"/"train").glob("*.png"))
print("mask unique sample:", np.unique(np.array(Image.open(p)))[:50])

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()


In [ ]:
NUM_CLASSES = 6
IGNORE = 255

model = torchvision.models.segmentation.deeplabv3_resnet50(
    weights=None,
    weights_backbone="DEFAULT",
    num_classes=NUM_CLASSES
).to(DEVICE)

criterion = nn.CrossEntropyLoss(ignore_index=IGNORE)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

EPOCHS = 35
total_iters = EPOCHS * len(train_loader)
base_lr = optimizer.param_groups[0]["lr"]

def poly_lr(iter_idx, total_iters, base_lr, power=0.9):
    return base_lr * ((1 - iter_idx / total_iters) ** power)

scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))

print("iters/epoch:", len(train_loader), "total_iters:", total_iters)


In [ ]:
@torch.no_grad()
def fast_confusion_matrix(pred, target, num_classes, ignore_index=255):
    mask = target != ignore_index
    pred = pred[mask]
    target = target[mask]
    if pred.numel() == 0:
        return torch.zeros((num_classes, num_classes), dtype=torch.int64, device=pred.device)
    k = (target * num_classes + pred).to(torch.int64)
    cm = torch.bincount(k, minlength=num_classes*num_classes).reshape(num_classes, num_classes)
    return cm

@torch.no_grad()
def evaluate(model, loader, num_classes=6, ignore_index=255):
    model.eval()
    cm_total = torch.zeros((num_classes, num_classes), dtype=torch.int64, device=DEVICE)
    loss_sum = 0.0
    n_batches = 0

    for imgs, masks in loader:
        imgs = imgs.to(DEVICE)
        masks = masks.to(DEVICE)

        out = model(imgs)["out"]
        loss = criterion(out, masks)
        loss_sum += loss.item()
        n_batches += 1

        pred = out.argmax(1)
        cm_total += fast_confusion_matrix(pred, masks, num_classes, ignore_index)

    tp = torch.diag(cm_total).float()
    fp = cm_total.sum(0).float() - tp
    fn = cm_total.sum(1).float() - tp
    denom = tp + fp + fn
    iou = torch.where(denom > 0, tp / denom, torch.zeros_like(denom))
    miou = iou.mean().item()

    return (loss_sum / max(n_batches,1)), miou, iou.detach().cpu().numpy()


In [ ]:
DRIVE_DIR = "/content/drive/MyDrive/deeplab6_runs"
import os, torch

def save_checkpoint(epoch, model, optimizer, scaler, global_iter, best_miou, path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    torch.save({
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "scaler_state": scaler.state_dict(),
        "global_iter": global_iter,
        "best_miou": best_miou,
        "img_size": IMG_SIZE,
        "num_classes": NUM_CLASSES
    }, path)


In [ ]:
!pip -q install tqdm

from tqdm.auto import tqdm
from pathlib import Path
import time, os
import torch
from google.colab import drive

# --- Mount Drive (idempotent) ---
drive.mount("/content/drive")

# --- Local + Drive dirs ---
RUN_DIR = Path("/content/runs_deeplab6")
RUN_DIR.mkdir(parents=True, exist_ok=True)

DRIVE_DIR = Path("/content/drive/MyDrive/deeplab6_runs")  # đổi tên folder nếu muốn
DRIVE_DIR.mkdir(parents=True, exist_ok=True)

best_path = RUN_DIR / "best.pt"
last_path = RUN_DIR / "last.pt"
drive_best = DRIVE_DIR / "best.pt"
drive_last = DRIVE_DIR / "last.pt"

def save_ckpt(path: Path, epoch: int, best_miou: float, global_iter: int):
    # full checkpoint to allow true resume
    torch.save({
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "scaler_state": scaler.state_dict(),
        "global_iter": global_iter,
        "best_miou": best_miou,
        "img_size": IMG_SIZE,
        "num_classes": NUM_CLASSES
    }, path)

best_miou = -1.0
global_iter = 0
LOG_EVERY = 50  # in log mỗi 50 step

for epoch in range(1, EPOCHS + 1):
    model.train()
    t0 = time.time()

    pbar = tqdm(enumerate(train_loader, start=1), total=len(train_loader), leave=True)
    running = 0.0

    for step, (imgs, masks) in pbar:
        imgs = imgs.to(DEVICE, non_blocking=True)
        masks = masks.to(DEVICE, non_blocking=True)

        lr = poly_lr(global_iter, total_iters, base_lr)
        for pg in optimizer.param_groups:
            pg["lr"] = lr

        optimizer.zero_grad(set_to_none=True)

        with torch.cuda.amp.autocast(enabled=(DEVICE == "cuda")):
            out = model(imgs)["out"]
            loss = criterion(out, masks)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        loss_item = loss.item()
        running += loss_item
        global_iter += 1

        pbar.set_description(f"Epoch {epoch}/{EPOCHS}")
        pbar.set_postfix({"lr": f"{lr:.2e}", "loss": f"{loss_item:.4f}", "avg": f"{running/step:.4f}"})

        if step % LOG_EVERY == 0:
            elapsed = time.time() - t0
            it_per_sec = step / max(elapsed, 1e-6)
            eta_sec = (len(train_loader) - step) / max(it_per_sec, 1e-6)
            print(f"[epoch {epoch:02d}] step {step:04d}/{len(train_loader)} "
                  f"lr={lr:.2e} loss={loss_item:.4f} avg={running/step:.4f} ETA={eta_sec/60:.1f}m")

    train_loss = running / max(len(train_loader), 1)

    # epoch-end eval
    val_loss, val_miou, _ = evaluate(model, val_loader, NUM_CLASSES, IGNORE)
    dt = time.time() - t0

    print(f"\nEpoch {epoch:02d}/{EPOCHS} DONE | {dt/60:.1f} min | "
          f"train_loss {train_loss:.4f} | val_loss {val_loss:.4f} | val_mIoU {val_miou:.4f}")

    # --- ALWAYS save LAST (local + drive) ---
    save_ckpt(last_path, epoch, best_miou, global_iter)
    os.system(f'cp "{last_path}" "{drive_last}"')

    # --- Save BEST (local + drive) if improved ---
    if val_miou > best_miou:
        best_miou = val_miou
        save_ckpt(best_path, epoch, best_miou, global_iter)
        os.system(f'cp "{best_path}" "{drive_best}"')
        print(" saved BEST ->", best_path, " | synced ->", drive_best)
    else:
        print(" synced LAST ->", drive_last)

print("Best val mIoU:", best_miou)
print("Local best:", best_path)
print("Drive best:", drive_best)
print("Drive last:", drive_last)


In [ ]:
from pathlib import Path
import torch
from google.colab import drive
import os, time
from tqdm.auto import tqdm

# 1) Mount Drive
drive.mount("/content/drive")

RUN_DIR = Path("/content/runs_deeplab6")
RUN_DIR.mkdir(parents=True, exist_ok=True)

DRIVE_DIR = Path("/content/drive/MyDrive/deeplab6_runs")
DRIVE_DIR.mkdir(parents=True, exist_ok=True)

drive_last = DRIVE_DIR / "last.pt"
drive_best = DRIVE_DIR / "best.pt"

# 2) Chọn checkpoint để resume (ưu tiên last)
ckpt_path = drive_last if drive_last.exists() else drive_best
assert ckpt_path.exists(), f"Không thấy checkpoint ở Drive: {drive_last} hoặc {drive_best}"

ckpt = torch.load(ckpt_path, map_location="cpu")


model.load_state_dict(ckpt["model_state"])
optimizer.load_state_dict(ckpt["optimizer_state"])
scaler.load_state_dict(ckpt["scaler_state"])

for st in optimizer.state.values():
    for k, v in st.items():
        if torch.is_tensor(v):
            st[k] = v.to(DEVICE)

start_epoch = int(ckpt["epoch"]) + 1
global_iter = int(ckpt.get("global_iter", 0))
best_miou = float(ckpt.get("best_miou", -1.0))

#sanity check config
print("Resume from:", ckpt_path)
print("start_epoch:", start_epoch, "| global_iter:", global_iter, "| best_miou:", best_miou)
print("ckpt img_size:", ckpt.get("img_size"), "| current IMG_SIZE:", IMG_SIZE)
print("ckpt num_classes:", ckpt.get("num_classes"), "| current NUM_CLASSES:", NUM_CLASSES)


In [ ]:
best_path = RUN_DIR / "best.pt"
last_path = RUN_DIR / "last.pt"
drive_best_out = DRIVE_DIR / "best.pt"
drive_last_out = DRIVE_DIR / "last.pt"

LOG_EVERY = 50

def save_ckpt(path: Path, epoch: int, best_miou: float, global_iter: int):
    torch.save({
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "scaler_state": scaler.state_dict(),
        "global_iter": global_iter,
        "best_miou": best_miou,
        "img_size": IMG_SIZE,
        "num_classes": NUM_CLASSES
    }, path)

for epoch in range(start_epoch, EPOCHS + 1):
    model.train()
    t0 = time.time()
    running = 0.0

    pbar = tqdm(enumerate(train_loader, start=1), total=len(train_loader), leave=True)

    for step, (imgs, masks) in pbar:
        imgs = imgs.to(DEVICE, non_blocking=True)
        masks = masks.to(DEVICE, non_blocking=True)

        lr = poly_lr(global_iter, total_iters, base_lr)
        for pg in optimizer.param_groups:
            pg["lr"] = lr

        optimizer.zero_grad(set_to_none=True)

        with torch.cuda.amp.autocast(enabled=(DEVICE == "cuda")):
            out = model(imgs)["out"]
            loss = criterion(out, masks)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        loss_item = float(loss.item())
        running += loss_item
        global_iter += 1

        pbar.set_description(f"Epoch {epoch}/{EPOCHS}")
        pbar.set_postfix({"lr": f"{lr:.2e}", "loss": f"{loss_item:.4f}", "avg": f"{running/step:.4f}"})

        if step % LOG_EVERY == 0:
            elapsed = time.time() - t0
            it_per_sec = step / max(elapsed, 1e-6)
            eta_sec = (len(train_loader) - step) / max(it_per_sec, 1e-6)
            print(f"[epoch {epoch:02d}] step {step:04d}/{len(train_loader)} "
                  f"lr={lr:.2e} loss={loss_item:.4f} avg={running/step:.4f} ETA={eta_sec/60:.1f}m")

    train_loss = running / max(len(train_loader), 1)

    val_loss, val_miou, _ = evaluate(model, val_loader, NUM_CLASSES, IGNORE)
    dt = time.time() - t0
    print(f"\nEpoch {epoch:02d}/{EPOCHS} DONE | {dt/60:.1f} min | "
          f"train_loss {train_loss:.4f} | val_loss {val_loss:.4f} | val_mIoU {val_miou:.4f}")

    # ALWAYS save LAST (local + drive)
    save_ckpt(last_path, epoch, best_miou, global_iter)
    os.system(f'cp "{last_path}" "{drive_last_out}"')

    # Save BEST if improved
    if val_miou > best_miou:
        best_miou = val_miou
        save_ckpt(best_path, epoch, best_miou, global_iter)
        os.system(f'cp "{best_path}" "{drive_best_out}"')
        print("saved BEST ->", best_path, "| synced ->", drive_best_out)
    else:
        print("synced LAST ->", drive_last_out)

print("Best val mIoU:", best_miou)
print("Drive best:", drive_best_out)
print("Drive last:", drive_last_out)


In [ ]:
# =========================
# CELL — Early Stopping (patience) + Auto Fine-tune from BEST
# Criterion: val_mIoU (maximize)
# =========================

from pathlib import Path
import os, time
import torch
from tqdm.auto import tqdm

# --- dirs (same as yours) ---
RUN_DIR = Path("/content/runs_deeplab6"); RUN_DIR.mkdir(parents=True, exist_ok=True)
DRIVE_DIR = Path("/content/drive/MyDrive/deeplab6_runs"); DRIVE_DIR.mkdir(parents=True, exist_ok=True)

local_last = RUN_DIR / "last.pt"
local_best = RUN_DIR / "best.pt"
drive_last = DRIVE_DIR / "last.pt"
drive_best = DRIVE_DIR / "best.pt"

def save_ckpt(path: Path, epoch: int, best_miou: float, global_iter: int):
    torch.save({
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "scaler_state": scaler.state_dict(),
        "global_iter": global_iter,
        "best_miou": best_miou,
        "img_size": IMG_SIZE,
        "num_classes": NUM_CLASSES
    }, path)

# -------------------------
# Early stopping config
# -------------------------
PATIENCE = 5          # stop if no improvement for 5 epochs
MIN_DELTA = 1e-4      # require improvement > min_delta
EARLY_STOP_MIN_EPOCH = 8  # don't early stop too early

# -------------------------
# Fine-tune config
# -------------------------
DO_FINE_TUNE = True
FT_EPOCHS = 8
FT_LR_SCALE = 0.25    # base_lr * 0.25
FT_PATIENCE = 3
FT_MIN_DELTA = 1e-4

LOG_EVERY = 50

# -------------------------
# Helpers
# -------------------------
def set_optimizer_lr(new_lr: float):
    for pg in optimizer.param_groups:
        pg["lr"] = new_lr

def load_ckpt_into_all(ckpt_path: Path):
    ckpt = torch.load(ckpt_path, map_location="cpu")
    model.load_state_dict(ckpt["model_state"])
    optimizer.load_state_dict(ckpt["optimizer_state"])
    scaler.load_state_dict(ckpt["scaler_state"])
    # move optimizer state to device
    for st in optimizer.state.values():
        for k, v in st.items():
            if torch.is_tensor(v):
                st[k] = v.to(DEVICE)
    return ckpt

# -------------------------
# Train one epoch
# -------------------------
def train_one_epoch(epoch: int, global_iter: int, total_iters: int, base_lr: float):
    model.train()
    t0 = time.time()
    running = 0.0

    pbar = tqdm(enumerate(train_loader, start=1), total=len(train_loader), leave=True)
    for step, (imgs, masks) in pbar:
        imgs = imgs.to(DEVICE, non_blocking=True)
        masks = masks.to(DEVICE, non_blocking=True)

        lr = poly_lr(global_iter, total_iters, base_lr)
        set_optimizer_lr(lr)

        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=(DEVICE == "cuda")):
            out = model(imgs)["out"]
            loss = criterion(out, masks)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        loss_item = float(loss.item())
        running += loss_item
        global_iter += 1

        pbar.set_description(f"Epoch {epoch}/{EPOCHS}")
        pbar.set_postfix({"lr": f"{lr:.2e}", "loss": f"{loss_item:.4f}", "avg": f"{running/step:.4f}"})

        if step % LOG_EVERY == 0:
            elapsed = time.time() - t0
            it_per_sec = step / max(elapsed, 1e-6)
            eta_sec = (len(train_loader) - step) / max(it_per_sec, 1e-6)
            print(f"[epoch {epoch:02d}] step {step:04d}/{len(train_loader)} "
                  f"lr={lr:.2e} loss={loss_item:.4f} avg={running/step:.4f} ETA={eta_sec/60:.1f}m")

    train_loss = running / max(len(train_loader), 1)
    return train_loss, global_iter, (time.time() - t0)

# =========================
# MAIN TRAIN with Early Stop
no_improve = 0
best_epoch = None

for epoch in range(start_epoch, EPOCHS + 1):
    train_loss, global_iter, dt_train = train_one_epoch(epoch, global_iter, total_iters, base_lr)

    val_loss, val_miou, _ = evaluate(model, val_loader, NUM_CLASSES, IGNORE)
    print(f"\nEpoch {epoch:02d}/{EPOCHS} DONE | {(dt_train)/60:.1f} min | "
          f"train_loss {train_loss:.4f} | val_loss {val_loss:.4f} | val_mIoU {val_miou:.4f}")

    # always save LAST
    save_ckpt(local_last, epoch, best_miou, global_iter)
    os.system(f'cp "{local_last}" "{drive_last}"')
    print("synced LAST ->", drive_last)

    # check improvement
    improved = (val_miou > best_miou + MIN_DELTA)
    if improved:
        best_miou = float(val_miou)
        best_epoch = epoch
        no_improve = 0
        save_ckpt(local_best, epoch, best_miou, global_iter)
        os.system(f'cp "{local_best}" "{drive_best}"')
        print("saved BEST ->", local_best, "| synced ->", drive_best)
    else:
        no_improve += 1
        print(f"no_improve={no_improve}/{PATIENCE} (best_mIoU={best_miou:.4f} at epoch {best_epoch})")

    # early stop condition
    if epoch >= EARLY_STOP_MIN_EPOCH and no_improve >= PATIENCE:
        print(f"EARLY STOP at epoch {epoch} (no improvement for {PATIENCE} epochs).")
        break

print("Train phase best_mIoU:", best_miou, "| best_epoch:", best_epoch)

# =========================
if DO_FINE_TUNE:
    assert drive_best.exists(), "Không thấy best.pt trên Drive để fine-tune."
    print("🔁 Fine-tune from BEST:", drive_best)

    ckpt = load_ckpt_into_all(drive_best)
    ft_start_epoch = int(ckpt["epoch"]) + 1
    ft_best = float(ckpt.get("best_miou", best_miou))
    ft_no_improve = 0

    ft_base_lr = base_lr * FT_LR_SCALE
    print(f"Fine-tune: base_lr {base_lr:.2e} -> {ft_base_lr:.2e} | epochs={FT_EPOCHS}")

    ft_total_iters = FT_EPOCHS * len(train_loader)
    ft_global_iter = 0  # reset schedule for fine-tune

    for i in range(FT_EPOCHS):
        epoch_ft = ft_start_epoch + i
        train_loss, ft_global_iter, dt_train = train_one_epoch(epoch_ft, ft_global_iter, ft_total_iters, ft_base_lr)

        val_loss, val_miou, _ = evaluate(model, val_loader, NUM_CLASSES, IGNORE)
        print(f"\n[FT] Epoch {epoch_ft:02d} DONE | {(dt_train)/60:.1f} min | "
              f"train_loss {train_loss:.4f} | val_loss {val_loss:.4f} | val_mIoU {val_miou:.4f}")

        save_ckpt(local_last, epoch_ft, ft_best, ft_global_iter)
        os.system(f'cp "{local_last}" "{drive_last}"')

        improved = (val_miou > ft_best + FT_MIN_DELTA)
        if improved:
            ft_best = float(val_miou)
            ft_no_improve = 0
            save_ckpt(local_best, epoch_ft, ft_best, ft_global_iter)
            os.system(f'cp "{local_best}" "{drive_best}"')
            print("[FT] saved BEST ->", drive_best)
        else:
            ft_no_improve += 1
            print(f"[FT] no_improve={ft_no_improve}/{FT_PATIENCE} (best_mIoU={ft_best:.4f})")

        if ft_no_improve >= FT_PATIENCE:
            print(f"EARLY STOP fine-tune (no improvement for {FT_PATIENCE} epochs).")
            break

    print("Fine-tune best_mIoU:", ft_best)
